# 1. Requirement Installation

In [ ]:
!apt install subversion
!pip install transformers
!pip install simpletransformers
!pip install datasets

import os
os.kill(os.getpid(), 9)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libapr1 libaprutil1 libserf-1-1 libsvn1
Suggested packages:
  db5.3-util libapache2-mod-svn subversion-tools
The following NEW packages will be installed:
  libapr1 libaprutil1 libserf-1-1 libsvn1 subversion
0 upgraded, 5 newly installed, 0 to remove and 34 not upgraded.
Need to get 2,237 kB of archives.
After this operation, 9,910 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libapr1 amd64 1.6.3-2 [90.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libaprutil1 amd64 1.6.1-2 [84.4 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libserf-1-1 amd64 1.3.9-6 [44.4 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd6

     |████████████████████████████████| 71kB 7.2MB/s 
  Found existing installation: tqdm 4.60.0
    Uninstalling tqdm-4.60.0:
      Successfully uninstalled tqdm-4.60.0


# 2. Imports

In [1]:
import os
import re
import tarfile
import shutil
import tarfile
import numpy as np
import pandas as pd
import unicodedata as ud

from google.colab import drive, files
from transformers import pipeline
from transformers import GPT2Tokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead, GPT2LMHeadModel
from simpletransformers.seq2seq import Seq2SeqModel
import pandas as pd
import math
import torch
from sklearn.model_selection import train_test_split
from tqdm import tqdm
device = 'cuda:0'

# 3. Data

## 3.1. Download Datasets

In [2]:
!svn checkout https://github.com/Alicia6N/movie_generator/trunk/datasets 


yes
A    datasets/anime_clean.csv
A    datasets/merged_dataset_descriptions.csv
A    datasets/merged_dataset_titles.csv
A    datasets/movies_metadata.csv
A    datasets/netflix_titles.csv
A    datasets/tmdb_5000_movies.csv
Checked out revision 5.


## 3.2. Tokenizer, Train and Test Datasets

In [3]:
merged_dataset = pd.read_csv('datasets/merged_dataset_titles.csv')
merged_dataset = merged_dataset.sample(frac=1).reset_index(drop=True)

def build_text_files(data, filename):
    f = open(filename, 'w')
    data = ''

    for row in merged_dataset['overview']:
        aux = str(row).strip()
        aux = re.sub(r"\s", " ", aux)
        data += aux + "  "
    f.write(data)

train, test = train_test_split(merged_dataset, test_size=0.25)
build_text_files(train, 'train_dataset.txt')
build_text_files(test, 'test_dataset.txt')

print("Train dataset length: ", len(train))
print("Test dataset length: ", len(test))

merged_dataset.head()

Train dataset length:  22336
Test dataset length:  7446


,title,overview
0,Jackie Brown,Jackie Brown is a flight attendant who gets ca...
1,Cruel Intentions 3,Two guys at a college prep school make wagers ...
2,Palindromes,"Aviva is thirteen, awkward and sensitive. Her ..."
3,Zandy's Bride,"Zandy Allan purchases a mail-order bride, Hann..."
4,Devo: Hardcore Live!,"Devo captured live in Oakland, performing earl..."


In [4]:
def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(tokenizer=tokenizer,
                                file_path=train_path,
                                block_size=128)
    
    test_dataset = TextDataset(tokenizer=tokenizer,
                               file_path=test_path,
                               block_size=128)
    
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, 
                                                    mlm=False)
    
    return train_dataset, test_dataset, data_collator

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


# 4. Bart Movie Title Generator Model

## 4.1. Model Configuration and Training

In [5]:
train.columns = ['target_text', 'input_text']
test.columns = ['target_text', 'input_text']
train_slice = train[0:20000]
print(len(train_slice))

20000


In [6]:
!mkdir cache

model_args = {
    "reprocess_input_data": False,
    "overwrite_output_dir": True,
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    'save_steps': -1,
    "max_seq_length": 100,
    "train_batch_size": 1,
    "num_train_epochs": 3,
    'cache_dir': 'cache/'
}

In [7]:
model = None
load_trained = True

if load_trained:
    #tar_name = 'trained_bart_base.tar'
    #tar_gdrive_id = '1N3p-4Ao9wTQ40cLiNjSxYVZvrdqZxTH8'

    #tar_name = 'trained_bart_base_2.tar'
    #tar_gdrive_id = '11NsCD_6CqJfmEk4ljAC4wtKhAiiPAPWG'

    #tar_name = 'trained_bart_base_1.tar'
    #tar_gdrive_id = '1A-r9t60DhbVYwHGGuJXHQ4ROZMNcTrNY'

    tar_name = 'trained_bart_base_3.tar'
    tar_gdrive_id = '13JtUXIbDuQFgifcDApGbisjuN5T-6isd'
    
    model_path = 'bart-model'

    try:
        os.mkdir(model_path)
    except OSError:
        print ("Creation of the directory %s failed" % model_path)

    print('Downloading finetuned model.')
    if not os.path.isfile(tar_name):
        !gdown --id {tar_gdrive_id}

    tar = tarfile.open(tar_name, "r:")
    tar.extractall()
    tar.close()

    !rm {tar_name}

    model = Seq2SeqModel(encoder_decoder_type="bart",
                        encoder_decoder_name='outputs')
else:
    model = Seq2SeqModel(encoder_decoder_type="bart",
                        encoder_decoder_name="facebook/bart-base",
                        args=model_args)

#model = Seq2SeqModel(encoder_decoder_type="bart", encoder_decoder_name="first_half_bart/", args=model_args)

Downloading...
From: https://drive.google.com/uc?id=13JtUXIbDuQFgifcDApGbisjuN5T-6isd
To: /content/trained_bart_base_3.tar
559MB [00:10, 52.9MB/s]


In [ ]:
if not load_trained:
    def save_checkpoint_to_gdrive(copy_folder=False):
        drive.mount('/content/drive')

        checkpoint_folder = os.path.join('trained_bart_base_3')

        if copy_folder:
            shutil.copytree(checkpoint_folder, "/content/drive/My Drive/" + checkpoint_folder)
        else:
            file_path = checkpoint_folder.replace(os.path.sep, '_') + '.tar'
            print(file_path)

            with tarfile.open(file_path, 'w') as tar:
                tar.add('outputs/config.json')
                tar.add('outputs/merges.txt')
                tar.add('outputs/model_args.json')
                tar.add('outputs/pytorch_model.bin')
                tar.add('outputs/special_tokens_map.json')
                tar.add('outputs/tokenizer_config.json')
                tar.add('outputs/training_args.bin')
                tar.add('outputs/vocab.json')

            shutil.copyfile(file_path, "/content/drive/My Drive/" + file_path)

In [ ]:
if not load_trained:
    model.train_model(train_slice)

    save_checkpoint_to_gdrive()

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)




Mounted at /content/drive
trained_bart_base_3.tar


In [ ]:
# Evaluate the model
result = model.eval_model(test)
print(result)

Output hidden; open in https://colab.research.google.com to view.

## 4.2. Title Generation

In [61]:
#plot = 'In the deep adamantinium mines on Alpha Centauri, the 21st century astronauts discover alien DNA. Most of the aliens have been surgically altered to look and feel like humans, and a few are still alive with an insatiable lust for blood.'
#plot = 'In the year 2077, the space wars have been won and Earth is ruled by Corporal Dante (Gavin Hood) a space-war veteran with a tough attitude. However, a new threat emerged that threatens to tear the planet apart. A mysterious Hound dog that seems to be bent on destroying everything in its path.'
#plot = 'The year is 2007, it is the future. The world is a harsh place. The only way to survive is to become a legend.  The story of the rise of the punk rock movement in Britain in the Seventies and Eighties.'
#plot = "In the deep titanium mines on Alpha Centauri, two peace-seeking miners – Jim Randle and Bucky Hardin – attempt to work together in the face of relentless radiation, alien beasts, and an irradiated planet from which there is no escape. A troubled young man and woman move into and go to a rundown apartment house."

predicted_title = model.predict([plot])

print(f'Predicted Title: {predicted_title}\n')
print(f'Description: {plot}\n\n\n')
#print(train[0:20000]['target_text'])



Predicted Title: ['The Mines of X']

Description: In the deep titanium mines on Alpha Centauri, two peace-seeking miners – Jim Randle and Bucky Hardin – attempt to work together in the face of relentless radiation, alien beasts, and an irradiated planet from which there is no escape. A troubled young man and woman move into and go to a rundown apartment house.





In [ ]:
test_sample = test.sample(n=20)

for idx, row in test_sample.iterrows():

    plot = row['input_text']
    true_title = row['target_text']

    # Predict with trained BART model
    predicted_title = model.predict([plot])[0]

    print(f'True Title: {true_title}\n')
    print(f'Predicted Title: {predicted_title}\n')
    print(f'Description: {plot}\n\n\n')


True Title: Ice Cream Man

Predicted Title: The Great Ice Cream Movie

Description: Poor Gregory, after being released from the Wishing Well Sanatorium, all he wants to do is make the children happy. So Gregory reopens the old ice cream factory, and all the unappreciative brats are reprocessed into the flavor of the day.






True Title: As Dreamers Do

Predicted Title: The Magic of Walt Disney

Description: The early life of Walt Disney is explored in this family film with an art house twist. Though his reality was often dark, it was skewed by his ever growing imagination and eternal optimism.






True Title: Sket

Predicted Title: The Boys Next Door

Description: When a young woman is cruelly and indiscriminately attacked by a notorious gang led by the violent Trey, her little 16 year old sister Kayla wants revenge and will stop at nothing to get it, even if it means joining a rival girl gang led by the volatile and damaged man-hating Danielle.






True Title: The Social Network

Predicted Title: Zuckerberg

Description: On a fall night in 2003, Harvard undergrad and computer programming genius Mark Zuckerberg sits down at his computer and heatedly begins working on a new idea. In a fury of blogging and programming, what begins in his dorm room as a small site among friends soon becomes a global social network and a revolution in communication. A mere six years and 500 million friends later, Mark Zuckerberg is the youngest billionaire in history... but for this entrepreneur, success leads to both personal and legal complications.






True Title: Lizzie

Predicted Title: Lizzie

Description: Lizzie, a young woman with amnesia, slowly comes to realize that she may be responsible for the horrific murder of her parents. After returning to the family home, Lizzie had terrifying flashbacks to her childhood, and comes face to face with the true horror of her past.





...


True Title: Les demoiselles de Rochefort

Predicted Title: La femme de l'amour

Description: Delphine and Solange are two sisters living in Rochefort. They are both looking for love, without being aware that their ideal partner is very close... A film where the scenario is much less important than its feeling of euphory, according to the director Jacques Demy.






True Title: Clerks - The Flying Car

Predicted Title: Trailer Jam

Description: Video short from Kevin Smith.  Dante and Randal are back and trapped in Dante's car during a very bad traffic jam while talking about a "flying car" from The Jetsons.






True Title: Virginia

Predicted Title: The Ice Case

Description: A sheriff sees his state senate bid slide out onto the ice when his daughter begins to date the son of a charming but psychologically disturbed woman with whom the sheriff has engaged in a two-decade-long affair.






True Title: Doom

Predicted Title: The Alien Factor

Description: A team of space marines known as the Rapid Response Tactical Squad, led by Sarge, is sent to a science facility on Mars after somebody reports a security breach. There, they learn that the alert came after a test subject, a mass murderer purposefully injected with alien DNA, broke free and began killing people. Dr. Grimm, who is related to team member Reaper, informs them all that the chromosome can mutate humans into monsters -- and is highly infectious.






True Title: Arlington Road

Predicted Title: The House Next Door

Description: Threats from sinister foreign nationals aren't the only thing to fear. Bedraggled college professor Michael Faraday has been vexed (and increasingly paranoid) since his wife's accidental death in a botched FBI operation. But all that takes a backseat when a seemingly all-American couple set up house next door.





In [ ]:
def generate_titles(input_file='movies_evaluation.csv', output_file='final_movies.csv', 
                    download=True):
  movies_eval = pd.read_csv(input_file)
  movies_eval.head()
  titles = []
  for index, row in movies_eval.iterrows():
    desc = row['Description']
    predicted_title = model.predict([desc])[0]
    titles.append(predicted_title)
  movies_eval['Title'] = titles
  movies_eval = movies_eval[['Title', 'Description', 'PPL',	'Tmp', 'TopK',	'TopP']]
  if download:
    print("Downloading file '{}'".format(output_file))
    movies_eval.to_csv(output_file, index = False, header=True)
    files.download("/content/final_movies.csv")
generate_titles()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>